# 實驗

In [1]:
import configparser
import os

from dotenv import load_dotenv

load_dotenv(".env")

CONFIG = configparser.ConfigParser()
PATH_CONFIG = os.getenv('path_2_config')

ENVIRONMENT = os.getenv('environment')
if ENVIRONMENT=="windows":
    CONFIG.read(PATH_CONFIG, encoding='utf-8')
else:
    CONFIG.read(PATH_CONFIG)

DEBUGGER = CONFIG["DEBUGGER"]["DEBUGGER"]

## 超參數設定


### 訓練參數

In [2]:
from utils.tools import get_file_name


path_data = CONFIG["datapath"]["Final_Quality"]
print(f"{path_data=}")
print(f"get_file_name(path_data)={get_file_name(path_data)}")
# path_prompt = CONFIG["datapath"]["init_os_prompt"]
# path_prompt = "D:\\實習\\evoprompt\\Ress\\dataset\\init_os_prompt_corpus_2024_0724_1357.json"
path_prompt = "D:\\實習\\evoprompt\\Ress\\dataset\\init_os_prompt_corpus.json"
type_llm = "Breeze"

# type_embedding = "multi-qa-mpnet-base-dot-v1"
type_embedding = "bgem3"
# type_embedding = None

path_data='D:\\實習\\evoprompt\\Ress\\dataset\\Final_Quality.json'
get_file_name(path_data)=Final_Quality


### 停止條件

In [3]:
path_stop_file = os.path.join(os.getcwd(), 'stop_true.txt') # 人工 early stop 的檔案位置
num_new_prompt = 20   # 必須是 prompt_population 的倍數
stop_score=100  # 練蠱終止條件(我是設超過baseline做100題後的分數，這邊看你資料量來設)

## 製作初始化資料

In [4]:
from main import init_setting
ttl_model, train_set, dev_set, ttl_pair_os_prompt_scores = init_setting(type_llm, type_embedding, path_data, path_prompt)

C:\Users\vicky\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


目前使用模型為:Breeze


 23%|██▎       | 10/44 [02:31<07:47, 13.74s/it]

In [ ]:
import json
from utils.tools import time_now

t = time_now()
path_folder = "D:\\實習\\evoprompt\\Ress\\init_prompt"
file_path = f"{path_folder}/{t}_20data.json"
data = {
    "corpus": get_file_name(path_data),
    "type_llm": type_llm,
    "type_embedding": type_embedding,
    "train_set": train_set,
    "dev_set": dev_set,
    "prompt_popularion": ttl_pair_os_prompt_scores
}
with open(file_path, 'w') as file:
    json.dump(data, file, indent=4)

## 初始化

In [10]:
import json

from utils.call_model.embedding import Encoder
from utils.call_model.llm import get_api

def get_init(path_init = None):
    
    if path_init==None:
        path_init = "D:\\實習\\evoprompt\\Ress\\init_prompt\\2024_0724_1430_20data.json"
    with open(path_init,  'r') as file:
        record_init = json.load(file)
    
    # 指定 llm
    get_llm_reply=get_api(record_init["type_llm"])

    # 指定 embedding model
    if record_init["type_embedding"] is None:
        embedding_model = None
    else:
        embedding_model = Encoder(record_init["type_embedding"])

    ttl_model = (get_llm_reply, embedding_model)
    
    # 指定訓練資料
    train_set = record_init["train_set"]
    dev_set = record_init["dev_set"]

    # 指定起始的 prompt
    prompt_scores = record_init["prompt_popularion"]
    
    return ttl_model, train_set, dev_set, prompt_scores

## ReSS

In [6]:
import json
from utils.ReSS import ReSS_short
from utils.tools import time_now

def test_ReSS():
    ttl_model, train_set, dev_set, ttl_pair_os_prompt_scores = get_init()

    stop_run_num = num_new_prompt   # 或是設一個回合數來終止(本來我會讓他跑到天荒地老所以沒有用for loop)

    record_population = []

    # record_population.append(ttl_pair_os_prompt_scores)
    sorted_pair = sorted(ttl_pair_os_prompt_scores,  key=lambda x: x['train_score'],  reverse=True)
    while(
        sorted_pair[0]['score']<stop_score
        and stop_run_num>0
        and not os.path.exists(path_stop_file)   # 人工 early stop
    ):
        new_population = ReSS_short(ttl_model, train_set, dev_set, sorted_pair)
        
        record_population.append(new_population)
        sorted_pair = sorted(new_population,  key=lambda x: x['train_score'],  reverse=True)

        stop_run_num -= 1

    # 儲存結果
    t = time_now()
    path_folder = CONFIG["datapath"]["record_folder"]
    file_path = f"{path_folder}\\experiment2\\{t}_ReSS.json"
    data = {
        "corpus": get_file_name(path_data),
        "type_llm": type_llm,
        "type_embedding": type_embedding,
        "best_promt": sorted_pair[0],
        "record": record_population
    }
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)
    print(f"\n\n\nthe result is saved at:\n{file_path}")

In [10]:
for i in range(1):
    test_ReSS()

目前使用模型為:Breeze


100%|██████████| 20/20 [05:24<00:00, 16.24s/it]





the result is saved at:
D:\實習\evoprompt\Ress\record\experiment2\2024_0725_0254_ReSS.json


## EvoDE

In [11]:
import json
from pathlib import Path
from utils.EvoPrompt import EvoPrompt_short
from utils.tools import time_now

def test_EvoDE():
    t = time_now()
    record_folder = CONFIG["datapath"]["record_folder"]
    path_folder = f"{record_folder}\\experiment2\\{t}_Evo"
    folder_path = Path(path_folder)
    folder_path.mkdir(parents=True, exist_ok=True)
    
    ttl_model, train_set, dev_set, ttl_pair_os_prompt_scores = get_init()
    
    path = "D:\\實習\\evoprompt\\Ress\\record\\experiment2\\2024_0725_1414_Evo\\01_Evo.json"
    with open(path, 'r', encoding='utf-8') as file:
        ttl_pair_os_prompt_scores = json.load(file)["record"]

    # training_data = training_data[:1]
    stop_run_num = int(num_new_prompt/4)   # 或是設一個回合數來終止(本來我會讓他跑到天荒地老所以沒有用for loop)
    # stop_run_num=4

    record_population = []

    record_population.append(ttl_pair_os_prompt_scores)
    sorted_pair = sorted(ttl_pair_os_prompt_scores,  key=lambda x: x['score'],  reverse=True)
    while(
        sorted_pair[0]['score']<stop_score
        and stop_run_num>0
        and not os.path.exists(path_stop_file)   # 人工 early stop
    ):
        new_population = EvoPrompt_short(ttl_model, train_set, dev_set, sorted_pair)
        
        record_population.append(new_population)
        sorted_pair = sorted(new_population,  key=lambda x: x['score'],  reverse=True)

        # 儲存結果
        file_path = f"{path_folder}\\{int(num_new_prompt/4) - stop_run_num+2:02d}_Evo.json"
        data = {
            "corpus": get_file_name(path_data),
            "type_llm": type_llm,
            "type_embedding": type_embedding,
            "best_promt": sorted_pair[0],
            "record": sorted_pair
        }
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"\n\n\nthe result is saved at:\n{file_path}")
        
        stop_run_num -= 1

    # # 儲存結果
    # t = time_now()
    # path_folder = CONFIG["datapath"]["record_folder"]
    # file_path = f"{path_folder}/{t}_ReSS.json"
    # data = {
    #     "corpus": get_file_name(path_data),
    #     "type_llm": type_llm,
    #     "type_embedding": type_embedding,
    #     "best_promt": sorted_pair[0],
    #     "record": record_population
    # }
    # with open(file_path, 'w') as file:
    #     json.dump(data, file, indent=4)
    

In [12]:
pre_file_path = None
for i in range(5):
    test_EvoDE()

目前使用模型為:Breeze


100%|██████████| 20/20 [07:35<00:00, 22.76s/it]





the result is saved at:
D:\實習\evoprompt\Ress\record\experiment2\2024_0725_1744_Evo\02_Evo.json


 15%|█▌        | 3/20 [01:17<07:19, 25.87s/it]


KeyboardInterrupt: 

## EvoGA

In [ ]:
import json
from pathlib import Path
from utils.EvoPrompt import EvoGA_short
from utils.tools import time_now

def test_EvoGA():
    t = time_now()
    record_folder = CONFIG["datapath"]["record_folder"]
    path_folder = f"{record_folder}\\experiment2\\{t}_EvoGA"
    folder_path = Path(path_folder)
    folder_path.mkdir(parents=True, exist_ok=True)
    
    ttl_model, train_set, dev_set, ttl_pair_os_prompt_scores = get_init()

    # training_data = training_data[:1]
    stop_run_num = int(num_new_prompt/4)   # 或是設一個回合數來終止(本來我會讓他跑到天荒地老所以沒有用for loop)
    # stop_run_num=4

    record_population = []

    record_population.append(ttl_pair_os_prompt_scores)
    sorted_pair = sorted(ttl_pair_os_prompt_scores,  key=lambda x: x['train_score'],  reverse=True)
    while(
        sorted_pair[0]['score']<stop_score
        and stop_run_num>0
        and not os.path.exists(path_stop_file)   # 人工 early stop
    ):
        new_population = EvoGA_short(ttl_model, train_set, dev_set, sorted_pair)
        
        record_population.append(new_population)
        sorted_pair = sorted(new_population,  key=lambda x: x['train_score'],  reverse=True)

        # 儲存結果
        file_path = f"{path_folder}\\{int(num_new_prompt/4) - stop_run_num+2:02d}_Evo.json"
        data = {
            "corpus": get_file_name(path_data),
            "type_llm": type_llm,
            "type_embedding": type_embedding,
            "best_promt": sorted_pair[0],
            "record": sorted_pair
        }
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"\n\n\nthe result is saved at:\n{file_path}")
        
        stop_run_num -= 1

    # # 儲存結果
    # t = time_now()
    # path_folder = CONFIG["datapath"]["record_folder"]
    # file_path = f"{path_folder}/{t}_ReSS.json"
    # data = {
    #     "corpus": get_file_name(path_data),
    #     "type_llm": type_llm,
    #     "type_embedding": type_embedding,
    #     "best_promt": sorted_pair[0],
    #     "record": record_population
    # }
    # with open(file_path, 'w') as file:
    #     json.dump(data, file, indent=4)
    